In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Paraíba - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [6]:
data = pd.read_csv('2003_mo_model_input_PB.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Consumo de Cimento (t)
0,2003-1,8.315958,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,118.926160,0.182304,0.656510,3.017292e+07,1.631334e+06,7.291169,2.736342e+07,38.426
1,2003-2,8.309895,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,117.724891,0.185909,0.656949,3.021010e+07,1.632950e+06,7.293438,2.737516e+07,31.672
2,2003-3,8.303831,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,118.501965,0.189593,0.657388,3.024728e+07,1.634566e+06,7.295708,2.738689e+07,28.775
3,2003-4,8.297767,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,119.390679,0.193353,0.657827,3.028445e+07,1.636182e+06,7.297978,2.739862e+07,28.571
4,2003-5,8.291704,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,121.517541,0.197186,0.658266,3.032163e+07,1.637798e+06,7.300247,2.741035e+07,31.613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294.165653,0.679715,NaN,NaN,NaN,NaN,NaN,95.860
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.457511,0.678961,NaN,NaN,NaN,NaN,NaN,90.790
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292.809622,0.677475,NaN,NaN,NaN,NaN,NaN,94.588
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,290.980478,0.676277,NaN,NaN,NaN,NaN,NaN,89.777


In [7]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
0,-0.800943,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.092830,-2.424081,-2.168844,-1.710694,-2.484291,-2.270808,-2.124240
1,-0.805366,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.140285,-2.392248,-2.133132,-1.692339,-2.436666,-2.234559,-2.091568
2,-0.809789,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.109587,-2.359718,-2.097421,-1.673984,-2.389041,-2.198310,-2.058896
3,-0.814212,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.074479,-2.326517,-2.061709,-1.655629,-2.341417,-2.162061,-2.026224
4,-0.818635,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.990458,-2.292670,-2.025998,-1.637274,-2.293792,-2.125812,-1.993552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.272641,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.885384,0.924040,1.235988,1.087341,-0.170948,0.675345,0.728937
188,1.277530,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.959697,0.934231,1.228601,1.077496,-0.163231,0.658514,0.715410
189,1.282419,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,2.094602,0.957607,1.221214,1.067651,-0.155514,0.641683,0.701882
190,1.287308,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,2.203705,0.987746,1.213827,1.057806,-0.147797,0.624852,0.688355


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      31.891
1      26.089
2      32.066
3      26.351
4      27.348
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Paraíba - Consumo de Cimento (t), Length: 240, dtype: float64

In [9]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
0,-0.800943,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.092830,-2.424081,-2.168844,-1.710694,-2.484291,-2.270808,-2.124240
1,-0.805366,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.140285,-2.392248,-2.133132,-1.692339,-2.436666,-2.234559,-2.091568
2,-0.809789,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.109587,-2.359718,-2.097421,-1.673984,-2.389041,-2.198310,-2.058896
3,-0.814212,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.074479,-2.326517,-2.061709,-1.655629,-2.341417,-2.162061,-2.026224
4,-0.818635,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.990458,-2.292670,-2.025998,-1.637274,-2.293792,-2.125812,-1.993552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.300589,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.747301,0.928050,1.431308,1.193935,-0.099354,0.924381,1.014355
158,1.292964,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.751028,0.933525,1.429336,1.195558,-0.105914,0.923232,1.010769
159,1.285339,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.748338,0.939401,1.427365,1.197181,-0.112475,0.922084,1.007184
160,1.277715,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.708461,0.946049,1.425393,1.198804,-0.119036,0.920935,1.003598


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      31.891
1      26.089
2      32.066
3      26.351
4      27.348
        ...  
157    58.962
158    73.459
159    59.150
160    64.833
161    63.840
Name: Paraíba - Consumo de Cimento (t), Length: 162, dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
162,1.262465,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,0.726315,0.936730,1.421451,1.202050,-0.132158,0.918637,0.996428
163,1.254841,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,0.750997,0.933298,1.419479,1.203673,-0.138719,0.917488,0.992842
164,1.247216,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,0.769881,0.933283,1.417508,1.205296,-0.145280,0.916339,0.989257
165,1.239591,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,0.777131,0.932533,1.415536,1.206919,-0.151841,0.915191,0.985671
166,1.231967,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,0.793980,0.931678,1.413565,1.208543,-0.158402,0.914042,0.982086
167,1.224342,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,0.777065,0.928951,1.411594,1.210166,-0.164963,0.912893,0.978500
168,1.216717,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,0.786100,0.926306,1.409622,1.211789,-0.171524,0.911744,0.974915
169,1.218526,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,0.771458,0.922846,1.399462,1.207161,-0.175977,0.901862,0.962308
170,1.220334,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,0.775076,0.919596,1.389302,1.202533,-0.180431,0.891981,0.949700
171,1.222142,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,0.761078,0.916956,1.379141,1.197906,-0.184884,0.882099,0.937093


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    61.721
163    69.240
164    70.048
165    67.172
166    64.220
167    63.021
168    62.639
169    53.839
170    56.652
171    53.408
172    50.543
173    64.699
174    60.219
175    64.217
176    57.591
177    64.307
178    61.971
179    57.480
180    68.053
181    55.231
182    53.723
183    57.457
184    62.755
185    48.148
186    62.190
187    62.219
188    61.329
189    81.988
190    79.341
191    76.646
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [13]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
#     train, train_val = validation_splitter(train_input, 7)
#     target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val, target_val),
                        validation_split=0.07,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [17]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3338745688, 3087408476, 2364554179, 666689671, 4036094267, 3803191913, 3352577067, 4167914819, 225680327, 1390433809, 1194511708, 1331593267, 2883884418, 2900680921, 3520223069, 1184473861, 1554391231, 1539697097, 3627971799, 2398163906, 2214948678, 1015090953, 2843841181, 3517400832, 2318076606]


Step: 0 ___________________________________________
val_loss: 36.38425827026367
winner_seed: 3338745688


Step: 1 ___________________________________________
val_loss: 56.37985610961914


Step: 2 ___________________________________________
val_loss: 106.8461685180664


Step: 3 ___________________________________________
val_loss: 343.1228942871094


Step: 4 ___________________________________________
val_loss: 889.9884643554688


Step: 5 ___________________________________________
val_loss: 2358.986328125


Step: 6 ___________________________________________
val_loss: 317.0262451171875


Step: 7 ___________________________________________
val_loss: 288.93560791015625


Step: 8 _____________

In [19]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 29ms/step - loss: 4399.2231 - val_loss: 16641.0020
Epoch 2/10000
5/5 [==============================] - 0s 7ms/step - loss: 5106.0923 - val_loss: 9025.3906
Epoch 3/10000
5/5 [==============================] - 0s 7ms/step - loss: 4021.1650 - val_loss: 3220.3352
Epoch 4/10000
5/5 [==============================] - 0s 7ms/step - loss: 3643.2112 - val_loss: 1628.6798
Epoch 5/10000
5/5 [==============================] - 0s 7ms/step - loss: 3064.3596 - val_loss: 421.6983
Epoch 6/10000
5/5 [==============================] - 0s 7ms/step - loss: 1475.9205 - val_loss: 1027.2563
Epoch 7/10000
5/5 [==============================] - 0s 7ms/step - loss: 999.8858 - val_loss: 51.5988
Epoch 8/10000
5/5 [==============================] - 0s 7ms/step - loss: 247.0626 - val_loss: 33.0362
Epoch 9/10000
5/5 [==============================] - 0s 7ms/step - loss: 166.3291 - val_loss: 43.3526
Epoch 10/10000
5/5 [==============================] - 0s 7ms/st

5/5 [==============================] - 0s 7ms/step - loss: 78.6784 - val_loss: 612.5465
Epoch 81/10000
5/5 [==============================] - 0s 7ms/step - loss: 75.3290 - val_loss: 677.4516
Epoch 82/10000
5/5 [==============================] - 0s 7ms/step - loss: 72.8131 - val_loss: 570.6199
Epoch 83/10000
5/5 [==============================] - 0s 7ms/step - loss: 75.9838 - val_loss: 572.2950
Epoch 84/10000
5/5 [==============================] - 0s 7ms/step - loss: 76.5303 - val_loss: 896.0635
Epoch 85/10000
5/5 [==============================] - 0s 7ms/step - loss: 76.3154 - val_loss: 711.0923
Epoch 86/10000
5/5 [==============================] - 0s 7ms/step - loss: 76.6020 - val_loss: 758.5078
Epoch 87/10000
5/5 [==============================] - 0s 8ms/step - loss: 71.9346 - val_loss: 884.5350
Epoch 88/10000
5/5 [==============================] - 0s 7ms/step - loss: 74.9657 - val_loss: 943.3680
Epoch 89/10000
5/5 [==============================] - 0s 7ms/step - loss: 98.7501 - val_

Epoch 159/10000
5/5 [==============================] - 0s 7ms/step - loss: 67.9798 - val_loss: 681.9469
Epoch 160/10000
5/5 [==============================] - 0s 8ms/step - loss: 66.4384 - val_loss: 547.0549
Epoch 161/10000
5/5 [==============================] - 0s 7ms/step - loss: 64.1732 - val_loss: 572.5684
Epoch 162/10000
5/5 [==============================] - 0s 7ms/step - loss: 70.9170 - val_loss: 517.1859
Epoch 163/10000
5/5 [==============================] - 0s 7ms/step - loss: 63.8781 - val_loss: 477.6942
Epoch 164/10000
5/5 [==============================] - 0s 7ms/step - loss: 77.3391 - val_loss: 557.6382
Epoch 165/10000
5/5 [==============================] - 0s 6ms/step - loss: 70.9562 - val_loss: 580.3812
Epoch 166/10000
5/5 [==============================] - 0s 7ms/step - loss: 69.3333 - val_loss: 708.1721
Epoch 167/10000
5/5 [==============================] - 0s 7ms/step - loss: 75.2235 - val_loss: 515.1901
Epoch 168/10000
5/5 [==============================] - 0s 8ms/st

5/5 [==============================] - 0s 7ms/step - loss: 75.6615 - val_loss: 481.1342
Epoch 238/10000
5/5 [==============================] - 0s 7ms/step - loss: 71.7350 - val_loss: 456.8993
Epoch 239/10000
5/5 [==============================] - 0s 6ms/step - loss: 70.4725 - val_loss: 327.2173
Epoch 240/10000
5/5 [==============================] - 0s 6ms/step - loss: 71.0091 - val_loss: 386.9530
Epoch 241/10000
5/5 [==============================] - 0s 6ms/step - loss: 74.1895 - val_loss: 403.3883
Epoch 242/10000
5/5 [==============================] - 0s 7ms/step - loss: 74.5930 - val_loss: 377.9859
Epoch 243/10000
5/5 [==============================] - 0s 7ms/step - loss: 64.5546 - val_loss: 371.9011
Epoch 244/10000
5/5 [==============================] - 0s 6ms/step - loss: 69.2286 - val_loss: 286.1267
Epoch 245/10000
5/5 [==============================] - 0s 6ms/step - loss: 69.5647 - val_loss: 255.3579
Epoch 246/10000
5/5 [==============================] - 0s 6ms/step - loss: 72.97

5/5 [==============================] - 0s 7ms/step - loss: 54.4349 - val_loss: 110.9269
Epoch 317/10000
5/5 [==============================] - 0s 7ms/step - loss: 44.3213 - val_loss: 52.9511
Epoch 318/10000
5/5 [==============================] - 0s 7ms/step - loss: 41.8082 - val_loss: 35.7379
Epoch 319/10000
5/5 [==============================] - 0s 8ms/step - loss: 46.7570 - val_loss: 50.9392
Epoch 320/10000
5/5 [==============================] - 0s 7ms/step - loss: 42.4035 - val_loss: 708.1318
Epoch 321/10000
5/5 [==============================] - 0s 7ms/step - loss: 46.7435 - val_loss: 1018.3501
Epoch 322/10000
5/5 [==============================] - 0s 7ms/step - loss: 46.7962 - val_loss: 1561.1986
Epoch 323/10000
5/5 [==============================] - 0s 7ms/step - loss: 43.1122 - val_loss: 2059.8303
Epoch 324/10000
5/5 [==============================] - 0s 7ms/step - loss: 50.3901 - val_loss: 779.3244
Epoch 325/10000
5/5 [==============================] - 0s 7ms/step - loss: 49.35

...

5/5 [==============================] - 0s 8ms/step - loss: 31.5011 - val_loss: 1437.6378
Epoch 709/10000
5/5 [==============================] - 0s 8ms/step - loss: 30.8749 - val_loss: 1350.7703
Epoch 710/10000
5/5 [==============================] - 0s 8ms/step - loss: 33.7209 - val_loss: 1447.8707
Epoch 711/10000
5/5 [==============================] - 0s 8ms/step - loss: 33.2275 - val_loss: 1310.1191
Epoch 712/10000
5/5 [==============================] - 0s 7ms/step - loss: 30.2230 - val_loss: 1311.0045
Epoch 713/10000
5/5 [==============================] - 0s 7ms/step - loss: 30.2657 - val_loss: 1342.7612
Epoch 714/10000
5/5 [==============================] - 0s 6ms/step - loss: 35.9088 - val_loss: 1400.0397
Epoch 715/10000
5/5 [==============================] - 0s 7ms/step - loss: 34.8278 - val_loss: 1392.0282
Epoch 716/10000
5/5 [==============================] - 0s 7ms/step - loss: 32.0420 - val_loss: 1314.4199
Epoch 717/10000
5/5 [==============================] - 0s 8ms/step - lo

5/5 [==============================] - 0s 7ms/step - loss: 31.9104 - val_loss: 1334.2910
Epoch 787/10000
5/5 [==============================] - 0s 8ms/step - loss: 35.3664 - val_loss: 1612.7943
Epoch 788/10000
5/5 [==============================] - 0s 7ms/step - loss: 31.7361 - val_loss: 1347.6956
Epoch 789/10000
5/5 [==============================] - 0s 7ms/step - loss: 33.3288 - val_loss: 1351.4532
Epoch 790/10000
5/5 [==============================] - 0s 7ms/step - loss: 31.0452 - val_loss: 1353.4082
Epoch 791/10000
5/5 [==============================] - 0s 7ms/step - loss: 39.4976 - val_loss: 1536.2852
Epoch 792/10000
5/5 [==============================] - 0s 7ms/step - loss: 32.1882 - val_loss: 1506.3639
Epoch 793/10000
5/5 [==============================] - 0s 7ms/step - loss: 32.4601 - val_loss: 1544.4633
Epoch 794/10000
5/5 [==============================] - 0s 7ms/step - loss: 33.6349 - val_loss: 1496.4152
Epoch 795/10000
5/5 [==============================] - 0s 8ms/step - lo

5/5 [==============================] - 0s 7ms/step - loss: 37.6486 - val_loss: 1662.6445
Epoch 865/10000
5/5 [==============================] - 0s 8ms/step - loss: 39.7691 - val_loss: 1749.5250
Epoch 866/10000
5/5 [==============================] - 0s 7ms/step - loss: 33.5481 - val_loss: 1813.2904
Epoch 867/10000
5/5 [==============================] - 0s 7ms/step - loss: 32.7919 - val_loss: 1874.9281
Epoch 868/10000
5/5 [==============================] - 0s 7ms/step - loss: 34.3704 - val_loss: 1714.7933
Epoch 869/10000
5/5 [==============================] - 0s 7ms/step - loss: 28.3670 - val_loss: 1616.4176
Epoch 870/10000
5/5 [==============================] - 0s 7ms/step - loss: 30.2183 - val_loss: 1617.1354
Epoch 871/10000
5/5 [==============================] - 0s 7ms/step - loss: 31.1265 - val_loss: 1812.0737
Epoch 872/10000
5/5 [==============================] - 0s 6ms/step - loss: 30.3891 - val_loss: 1820.0566
Epoch 873/10000
5/5 [==============================] - 0s 8ms/step - lo

5/5 [==============================] - 0s 7ms/step - loss: 26.4084 - val_loss: 1524.8472
Epoch 943/10000
5/5 [==============================] - 0s 7ms/step - loss: 30.5043 - val_loss: 1338.3390
Epoch 944/10000
5/5 [==============================] - 0s 7ms/step - loss: 30.0974 - val_loss: 1529.0732
Epoch 945/10000
5/5 [==============================] - 0s 7ms/step - loss: 30.6393 - val_loss: 1379.1279
Epoch 946/10000
5/5 [==============================] - 0s 7ms/step - loss: 33.7461 - val_loss: 1333.2509
Epoch 947/10000
5/5 [==============================] - 0s 7ms/step - loss: 31.8762 - val_loss: 1344.9806
Epoch 948/10000
5/5 [==============================] - 0s 7ms/step - loss: 30.1233 - val_loss: 1368.7054
Epoch 949/10000
5/5 [==============================] - 0s 7ms/step - loss: 28.7504 - val_loss: 1412.9492
Epoch 950/10000
5/5 [==============================] - 0s 7ms/step - loss: 29.7985 - val_loss: 1363.0436
Epoch 951/10000
5/5 [==============================] - 0s 7ms/step - lo

Epoch 1020/10000
5/5 [==============================] - 0s 7ms/step - loss: 29.4197 - val_loss: 2192.4187
Epoch 1021/10000
5/5 [==============================] - 0s 7ms/step - loss: 28.2354 - val_loss: 2038.3004
Epoch 1022/10000
5/5 [==============================] - 0s 7ms/step - loss: 34.9314 - val_loss: 2244.7988
Epoch 1023/10000
5/5 [==============================] - 0s 8ms/step - loss: 34.7720 - val_loss: 2228.0142
Epoch 1024/10000
5/5 [==============================] - 0s 8ms/step - loss: 32.1910 - val_loss: 2023.0586
Epoch 1025/10000
5/5 [==============================] - 0s 8ms/step - loss: 34.1582 - val_loss: 1835.4386
Epoch 1026/10000
5/5 [==============================] - 0s 8ms/step - loss: 32.1365 - val_loss: 2061.3318
Epoch 1027/10000
5/5 [==============================] - 0s 7ms/step - loss: 29.1340 - val_loss: 1997.1396
Epoch 1028/10000
5/5 [==============================] - 0s 7ms/step - loss: 30.4204 - val_loss: 1884.0278
Epoch 1029/10000
5/5 [========================

In [20]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [21]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,58.010674,56.281158,56.054428,55.564255,52.551437,49.811447,39.715195,38.651131,38.898132,39.590118,36.523102,48.932991,36.97216,33.383018,37.866478,40.76004,43.972759,33.386143,29.533875,31.392685,32.231926,29.122335,24.162369,22.392145,24.230759,33.933983,48.56424,61.095856,76.48497,79.722137
Target,61.721,69.24,70.048,67.172,64.22,63.021,62.639,53.839,56.652,53.408,50.543,64.699,60.219,64.217,57.591,64.307,61.971,57.48,68.053,55.231,53.723,57.457,62.755,48.148,62.19,62.219,61.329,81.988,79.341,76.646
Error,3.710327,12.958839,13.993568,11.607742,11.668564,13.209553,22.923805,15.18787,17.753868,13.817883,14.019897,15.766006,23.246841,30.833984,19.724522,23.546959,17.998241,24.093857,38.519127,23.838314,21.491074,28.334665,38.592632,25.755854,37.95924,28.285019,12.764759,20.892143,2.856033,3.076134


In [22]:
display(mae)
display(mape)

19.614244

0.3227207

In [23]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [24]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 17ms/step
Ano-0: |Prediction[[570.58405]] - Target[737.202]| =  Error: [[166.61798]]; MAPE:[[0.226014]]
1/1 [==============================] - 0s 18ms/step
Ano-0: |Prediction[[395.17596]] - Target[711.152]| =  Error: [[315.976]]; MAPE:[[0.44431573]]
1/1 [==============================] - 0s 19ms/step
Ano-5: |Prediction[[324.03192]] - Target[423.713]| =  Error: [[99.68109]]; MAPE:[[0.23525615]]


[array([[166.61798]], dtype=float32),
 array([[315.976]], dtype=float32),
 array([[99.68109]], dtype=float32)]

194.09169

0.30186197